In [1]:
import os
import numpy as np
from arch import arch_model
import math
import datetime
from datetime import datetime
from scipy.stats import norm
import pandas as pd
import pickle

In [2]:
def save(data , fn):
    with open(fn, 'wb') as f:
        pickle.dump(data, f)
def load(fn): 
    with open(fn, 'rb') as f:
        data = pickle.load(f)
        f.close()
    return data

In [3]:
class EuropeanPut:

    def put_delta(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
            ):
        b = math.exp(-risk_free_rate*time_to_expiration)
        x1 = math.log(asset_price/(b*strike_price)) + .5*(asset_volatility*asset_volatility)*time_to_expiration
        x1 = x1/(asset_volatility*(time_to_expiration**.5))
        z1 = norm.cdf(x1)
        return z1 - 1

    def put_price(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
            ):
        b = math.exp(-risk_free_rate*time_to_expiration)
        x1 = math.log((b*strike_price)/asset_price) + .5*(asset_volatility*asset_volatility)*time_to_expiration
        x1 = x1/(asset_volatility*(time_to_expiration**.5))
        z1 = norm.cdf(x1)
        z1 = b*strike_price*z1
        x2 = math.log((b*strike_price)/asset_price) - .5*(asset_volatility*asset_volatility)*time_to_expiration
        x2 = x2/(asset_volatility*(time_to_expiration**.5))
        z2 = norm.cdf(x2)
        z2 = asset_price*z2
        return z1 - z2

    def __init__(
        self, asset_price, asset_volatility, strike_price,
        time_to_expiration, risk_free_rate
            ):
        self.asset_price = asset_price
        self.asset_volatility = asset_volatility
        self.strike_price = strike_price
        self.time_to_expiration = time_to_expiration
        self.risk_free_rate = risk_free_rate
        self.price = self.put_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
        self.delta = self.put_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)

In [4]:
def hedge(vol, S):
    initial_sp = EuropeanPut(prices[0], vol/100, S, total.days/365, 0.06)
    hedge_deltas = []
    sp_deltas = []
    opt_price = []
    for i in range(0,len(days)):
        put = EuropeanPut(prices[i], initial_sp.asset_volatility, initial_sp.strike_price, time[i]/365, initial_sp.risk_free_rate)
        hedge_deltas.append(put.delta)    
        sp_deltas.append(-1*put.delta)
        opt_price.append(put.price)
    sp_option_price = initial_sp.price
    premium = sp_option_price*total_share # ค่า premium = option price x จำนวนหุ้นทั้งหมด
    DeltaxShare = round(total_share*sp_deltas[0]) # จำนวนหุ้นที่จะshort เพื่อhedgeในวันแรก
    DeltaxShare = (DeltaxShare // 100) * 100
    cumulative_cost = round(DeltaxShare*prices[0]) # เงินที่ได้จากการshortหุ้นเพื่อ hedge ในวันแรก
    DeltaxShares = []
    cumulative_costs = []
    DeltaxShares.append(DeltaxShare)
    cumulative_costs.append(cumulative_cost)
    
    for i in range(1,len(days)):    
        DeltaxShare = round(total_share*sp_deltas[i])
        DeltaxShare = (DeltaxShare // 100) * 100
        d_share_held = DeltaxShare-DeltaxShares[i-1] # เช็คว่าปริมาณหุ้นที่เราจะshortเพื่อ hedge ของวันนี้ มากกว่าหรือน้อยกว่าปริมาณหุ้นเมื่อวาน
        if d_share_held < 0: # ถ้าผลต่างเป็นลบ แปลว่าเราจะซื้อหุ้นมาคืนบางส่วน
            if d_share_held <= -100: # เช็คว่าหุ้นที่จะซื้อมาคืน มีปริมาณมากกว่า100หรือไม่
                cost = prices[i]*d_share_held
                cumulative_cost = cumulative_cost+cost # เงินที่ใช้ในการซื้อหุ้นคืน จะนำไปหักล้างกับค่า cumulative
            elif d_share_held > -100: # ถ้าปริมาณหุ้นไม่ถึง 100 จะไม่เกิดการซื้อหุ้นมาคืน hedge delta จะเท่าเดิมไม่เปลี่ยนแปลง
                hedge_deltas[i] = hedge_deltas[i-1]
    #         cumulative_cost = cumulative_cost+funding_interests[i-1]
        elif d_share_held > 0: # ถ้าผลต่างเป็นบวก แปลว่าเราจะshortหุ้นเพิ่ม
            if d_share_held > 100: # เช็คว่าหุ้นที่จะshort มีปริมาณมากกว่า100หรือไม่
                cost = prices[i]*d_share_held
                cumulative_cost = cumulative_cost+cost # เงินที่ได้จากการshortหุ้น จะนำไปรวมกับ cumulative
            elif d_share_held < 100: # ถ้าปริมาณหุ้นไม่ถึง 100 จะไม่เกิดการshort hedge delta จะเท่าเดิมไม่เปลี่ยนแปลง
                hedge_deltas[i] = hedge_deltas[i-1]
    #         cumulative_cost = cumulative_cost+funding_interests[i-1]
        DeltaxShare = round(total_share*hedge_deltas[i]*-1)
        DeltaxShare = (DeltaxShare // 100) * 100
        DeltaxShares.append(DeltaxShare)
        cumulative_costs.append(round(cumulative_cost))
    
    if prices[len(prices)-1] < initial_sp.strike_price - sp_option_price: # ถ้าราคาของหุ้นอ้างอิงในวันครบกำหนดน้อยกว่า ค่าของstrike priceลบกับค่าoption price. option จะมีสถานะ OTM
        # status = "OTM"
        PandL = (-initial_sp.strike_price * total_share) + (total_share - DeltaxShares[len(DeltaxShares)-1]) * prices[len(prices)-1] + premium + cumulative_costs[len(cumulative_costs)-1]
    #p&lคิดจาก -1*ราคาstrike priceคูณกับปริมาณหุ้นตามสัญญา + เงินที่ได้จากหุ้นที่เหลือ หลังนำหุ้นที่ได้จากสัญญาไปคืนให้ตามปริมาณที่shortมา +premium+cumulative ที่สะสมมาทั้งหมด
    elif prices[len(prices)-1] >= initial_sp.strike_price - sp_option_price: # ถ้าราคาของหุ้นอ้างอิงในวันครบกำหนดมากกว่า ค่าของstrike priceลบกับค่าoption price. option จะมีสถานะ ITM
        # status = "ITM"
        PandL = premium + cumulative_costs[len(cumulative_costs)-1]- (DeltaxShares[len(DeltaxShares)-1]*prices[len(prices)-1])
    #p&lคิดจากpremium+cumulative ที่สะสมมาทั้งหมด                       ลบกับปริมาณหุ้นที่ต้องซื้อมาคืนคูณกับราคาของหุ้นอ้างอิงวันสุดท้าย
    return PandL

In [5]:
SOURCE_DIR = '../SET50 JAN_JULY/'
quotes ={}
for i in os.listdir(SOURCE_DIR):
    if i.endswith('.pkl'):
        quotes[os.path.splitext(i)[0]] = None

In [6]:
count = 0
contract = 10000 # มี option ทั้งหมด 10,000 สัญญา
share_per_contract = 100 # สัญญาละ 100 หุ้น
total_share = share_per_contract*contract

for a in os.listdir(SOURCE_DIR):
    
    df =  pd.DataFrame(columns=['start option day', 'volatility * iteration', 'profit and loss'])
    if a.endswith('.pkl'):
        path = os.path.join(SOURCE_DIR, a)
        name = os.path.splitext(a)[0]
        data = load(path)
#         count += 1
    
    #option start day
    start_day = []
    check1 = 0
    for i in data['TRADE'].keys():
        if i == '20190301':
            check1 = 1    
        if check1 != 1:
            continue
        if i == '20190502':
            break
        start_day.append(i)
    
    for i in start_day:

        #find historical volatility
        histo = []
        dayy = []
        for j in data['TRADE'].keys():
            if j == i:
                break
            histo.append(data['TRADE'][j]["Price"].iloc[-1])
            dayy.append(j)
        his_price =  pd.DataFrame(
        {"historical price" : histo, 
        }, 
        index = dayy)
        his_price['Return'] = his_price['historical price'].pct_change()
        his_price['Return'] = 100*np.log(1 + his_price['Return'])
        data2 = his_price['Return'].to_numpy()
        data2 = data2[np.logical_not(np.isnan(data2))]
        n_test = 1
        train, test = data2[:-n_test], data2[-n_test:]
        model = arch_model(train, mean='Zero', vol='GARCH', p=1, q=1)
        model_fit = model.fit()
        predictions = model_fit.forecast(horizon=n_test)
        histo_vol = float("{:.2f}".format(float((predictions.variance.values[-1, :])**0.5 * np.sqrt(252))))

        #ข้อมูลราคาหุ้นในช่วงเวลาของ option
        days = []
        prices = []
        check2 = 0
        day_check = 66
        for j in data['TRADE'].keys():
            if day_check <= 0:
                break
            if j == i:
                check2 = 1    
            if check2 != 1:
                continue
            prices.append(data['TRADE'][j]["Price"].iloc[-1])
            days.append(j)
            day_check -= 1

        #จัดการข้อมูลเกี่ยวกับระยะเวลา
        days_new = []
        for j in days:
            time = f'{j[0:4]}-{j[4:6]}-{j[6:]}'
            date_time_obj = datetime.strptime(time, '%Y-%m-%d')
            days_new.append(date_time_obj)   
        start_date = days_new[0]
        ex = days_new[len(days)-1]
        total = ex-start_date # จำนวนวันทั้งหมด ตั้งแต่วันแรกถึงวันคบกำหนด
        time = []
        for j in range(0,len(days_new)):
            date = days_new[j]
            t = ex-date
            if t.days==0 :
                time.append(1)
                continue
            time.append(t.days)

        #hedging
        start_price = prices[0]
        strike = start_price * 0.95
        begin_option = []
        vols = []
        pandl = []
        ITERATION = 1
        while(ITERATION <= 2):
            pl = hedge(float("{:.3f}".format(histo_vol*ITERATION)), strike)
            begin_option.append(i)
            vols.append(float("{:.3f}".format(histo_vol*ITERATION)))
            pandl.append(round(pl))
            ITERATION = ITERATION + 0.05
            ITERATION = float("{:.2f}".format(float(ITERATION)))
        
        #dataframe to dict
        df_sp =  pd.DataFrame(
        {"start option day" : begin_option,
         "volatility * iteration" : vols,
         "profit and loss" : pandl,
        }, 
        )
        df = df.append(df_sp)

    df = df.reset_index()
    df = df.drop('index', axis=1)
    df_dict = df.to_dict()
        
    quotes[name] = df_dict
        
    if count >= 1:
        break

Iteration:      1,   Func. Count:      5,   Neg. LLF: 96.43351458064902
Iteration:      2,   Func. Count:     11,   Neg. LLF: 62.63732038640057
Iteration:      3,   Func. Count:     15,   Neg. LLF: 62.636839754451124
Iteration:      4,   Func. Count:     19,   Neg. LLF: 62.6344063539312
Iteration:      5,   Func. Count:     23,   Neg. LLF: 62.633342881051874
Iteration:      6,   Func. Count:     27,   Neg. LLF: 62.633382286187405
Iteration:      7,   Func. Count:     32,   Neg. LLF: 62.6328727319081
Iteration:      8,   Func. Count:     36,   Neg. LLF: 62.632872173809886
Optimization terminated successfully    (Exit mode 0)
            Current function value: 62.632872173809886
            Iterations: 8
            Function evaluations: 36
            Gradient evaluations: 8


/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      5,   Neg. LLF: 99.01747895183448
Iteration:      2,   Func. Count:     11,   Neg. LLF: 63.75080488413391
Iteration:      3,   Func. Count:     15,   Neg. LLF: 63.75056583499756
Iteration:      4,   Func. Count:     19,   Neg. LLF: 63.74968593749165
Iteration:      5,   Func. Count:     23,   Neg. LLF: 63.74895304904199
Iteration:      6,   Func. Count:     27,   Neg. LLF: 63.7482900193543
Iteration:      7,   Func. Count:     31,   Neg. LLF: 63.74808222379504
Iteration:      8,   Func. Count:     35,   Neg. LLF: 63.748069922048714
Iteration:      9,   Func. Count:     38,   Neg. LLF: 63.7480699221122
Optimization terminated successfully    (Exit mode 0)
            Current function value: 63.748069922048714
            Iterations: 9
            Function evaluations: 38
            Gradient evaluations: 9


/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      5,   Neg. LLF: 95.56553226187368
Iteration:      2,   Func. Count:     11,   Neg. LLF: 64.85179203304749
Iteration:      3,   Func. Count:     15,   Neg. LLF: 64.85172251919985
Iteration:      4,   Func. Count:     19,   Neg. LLF: 64.8516075443314
Iteration:      5,   Func. Count:     23,   Neg. LLF: 64.85141362672778
Iteration:      6,   Func. Count:     27,   Neg. LLF: 64.85114962894039
Iteration:      7,   Func. Count:     31,   Neg. LLF: 64.85070428684287
Iteration:      8,   Func. Count:     35,   Neg. LLF: 64.85060142637016
Iteration:      9,   Func. Count:     39,   Neg. LLF: 64.85056819428347
Iteration:     10,   Func. Count:     42,   Neg. LLF: 64.85056819436375
Optimization terminated successfully    (Exit mode 0)
            Current function value: 64.85056819428347
            Iterations: 10
            Function evaluations: 42
            Gradient evaluations: 10
Iteration:      1,   Func. Count:      5,   Neg. LLF: 94.59566289499226


...

/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      5,   Neg. LLF: 177.38128894014994
Iteration:      2,   Func. Count:      9,   Neg. LLF: 176.7565311161624
Iteration:      3,   Func. Count:     13,   Neg. LLF: 175.9648765474857
Iteration:      4,   Func. Count:     17,   Neg. LLF: 175.9613611165017
Iteration:      5,   Func. Count:     21,   Neg. LLF: 175.95864534812526
Iteration:      6,   Func. Count:     25,   Neg. LLF: 175.95862905630813
Iteration:      7,   Func. Count:     28,   Neg. LLF: 175.95862927295013
Optimization terminated successfully    (Exit mode 0)
            Current function value: 175.95862905630813
            Iterations: 7
            Function evaluations: 28
            Gradient evaluations: 7
Iteration:      1,   Func. Count:      5,   Neg. LLF: 179.20204898274196
Iteration:      2,   Func. Count:      9,   Neg. LLF: 178.47921441416318
Iteration:      3,   Func. Count:     13,   Neg. LLF: 177.6812420499775
Iteration:      4,   Func. Count:     17,   Neg. LLF: 177.67716265

/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(
/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


Iteration:      1,   Func. Count:      5,   Neg. LLF: 182.7717840609593
Iteration:      2,   Func. Count:      9,   Neg. LLF: 181.82882623828718
Iteration:      3,   Func. Count:     13,   Neg. LLF: 181.31326664677906
Iteration:      4,   Func. Count:     17,   Neg. LLF: 181.1833864240484
Iteration:      5,   Func. Count:     21,   Neg. LLF: 181.05877623042517
Iteration:      6,   Func. Count:     25,   Neg. LLF: 181.0416701235454
Iteration:      7,   Func. Count:     29,   Neg. LLF: 181.03883267430103
Iteration:      8,   Func. Count:     33,   Neg. LLF: 181.03879401052725
Iteration:      9,   Func. Count:     36,   Neg. LLF: 181.03879424495832
Optimization terminated successfully    (Exit mode 0)
            Current function value: 181.03879401052725
            Iterations: 9
            Function evaluations: 36
            Gradient evaluations: 9


/home/pleum/.local/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [7]:
import json

with open("../output/sp_output.json", "w") as outfile:
    json.dump(quotes, outfile)